In [32]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, to_date

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/04 03:07:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/04 03:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/04 03:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
26/02/04 03:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
spark.sql("""select count(1) from bronze.bsc_mainnet_transactions""").show(truncate=False)

+--------+
|count(1)|
+--------+
|1503099 |
+--------+



In [ ]:
# Arrow 加速
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")
# spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")

'true'

In [ ]:
# !pip install PyArrow
# Spark DataFrame -> Arrow Columnar Batch（列式内存）（零拷贝 / 极少拷贝）-> pandas.DataFrame

In [53]:
import pandas as pd

pd.set_option("display.max_colwidth", 50) # by default 50

In [ ]:
spark.sql("""select * from bronze.bsc_mainnet_transactions limit 5""").toPandas()

,block_height,job_name,run_id,raw,kafka_topic,kafka_partition,kafka_offset,kafka_timestamp,kafka_date
0,79065609,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,"{""blockHash"": ""0x3ec97cf0fb6a47660f4c6cf527635...",blockchain.bsc.ingestion.transactions.raw,3,48947,2026-02-03 13:25:20.016,2026-02-03
1,79065609,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,"{""blockHash"": ""0x3ec97cf0fb6a47660f4c6cf527635...",blockchain.bsc.ingestion.transactions.raw,3,48948,2026-02-03 13:25:20.033,2026-02-03
2,79065609,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,"{""blockHash"": ""0x3ec97cf0fb6a47660f4c6cf527635...",blockchain.bsc.ingestion.transactions.raw,3,48949,2026-02-03 13:25:20.033,2026-02-03
3,79065609,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,"{""blockHash"": ""0x3ec97cf0fb6a47660f4c6cf527635...",blockchain.bsc.ingestion.transactions.raw,3,48950,2026-02-03 13:25:20.033,2026-02-03
4,79065609,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,"{""blockHash"": ""0x3ec97cf0fb6a47660f4c6cf527635...",blockchain.bsc.ingestion.transactions.raw,3,48951,2026-02-03 13:25:20.033,2026-02-03


In [14]:
df = spark.sql("""select raw from bronze.bsc_mainnet_transactions limit 10""")
# 0x4b67a6d

In [43]:
import json

df = spark.sql("""
select raw
from bronze.bsc_mainnet_transactions
limit 1
""").toPandas()

obj = json.loads(df.loc[0, "raw"])
obj

{'blockHash': '0x732acb841055b58d61c361c1b850868a0133c833b88c6ffb0a00fd5459a38e8c',
 'blockNumber': '0x4b672f0',
 'from': '0xfab408ade33026cac90759e1a2e8f1e3b0655428',
 'gas': '0x927c0',
 'gasPrice': '0x5f5e100',
 'hash': '0x7ec1d3c52243ca42bcf187fd0e212b67206e22bb157deede9e83d1c49bf79584',
 'input': '0xfb3bdb4100000000000000000000000000000000000000000000000f8612e7b0235d80000000000000000000000000000000000000000000000000000000000000000080000000000000000000000000e341bda6efa3a9cec138ea51f34d81dd77054970000000000000000000000000000000000000000000000000000000006981f9690000000000000000000000000000000000000000000000000000000000000002000000000000000000000000bb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c00000000000000000000000066969ecd173451f00a4652a53acc6246569d4444',
 'nonce': '0x255',
 'to': '0x10ed43c718714eb63d5aa57b78b54704e256024e',
 'transactionIndex': '0x1',
 'value': '0x8f98e611f6aad',
 'type': '0x1',
 'accessList': [],
 'chainId': '0x38',
 'v': '0x0',
 'r': '0x5e97fcaa064ae578e36d1dcdc3f334

In [24]:
# spark.sql("""select * from bronze.bsc_mainnet_transactions limit 10""").show(truncate=False)

In [16]:
from pyspark.sql.types import *

tx_schema = StructType([
    StructField("blockHash", StringType()),
    StructField("blockNumber", StringType()),
    StructField("from", StringType()),
    StructField("to", StringType()),
    StructField("nonce", StringType()),
    StructField("hash", StringType()),
    StructField("transactionIndex", StringType()),
    StructField("value", StringType()),
    StructField("gas", StringType()),
    StructField("gasPrice", StringType()),
    StructField("maxFeePerGas", StringType()),
    StructField("maxPriorityFeePerGas", StringType()),
    StructField("input", StringType()),
    StructField("type", StringType()),
    StructField("accessList", ArrayType(
        StructType([
            StructField("address", StringType()),
            StructField("storageKeys", ArrayType(StringType()))
        ])
    )),
    StructField("chainId", StringType()),
    StructField("v", StringType()),
    StructField("r", StringType()),
    StructField("s", StringType()),
    StructField("yParity", StringType()),
])

In [17]:
from pyspark.sql import functions as F

# read bronze table
df_bronze = spark.table("bronze.bsc_mainnet_transactions")

# parse raw json -> struct
df_parsed = (
    df_bronze
    .withColumn("tx", F.from_json(F.col("raw"), tx_schema))
)

In [47]:
# explode column
silver_tx = (
    df_parsed
    .select(
        # ----------------
        # Kafka & ingestion metadata
        # ----------------
        "block_height",
        "job_name",
        "run_id",
        "kafka_topic",
        "kafka_partition",
        "kafka_offset",
        "kafka_timestamp",
        "kafka_date",

        # ----------------
        # Transaction identity
        # ----------------
        F.col("tx.hash").alias("tx_hash"),
        F.col("tx.blockHash").alias("block_hash"),
        F.col("tx.from").alias("from_address"),
        F.col("tx.to").alias("to_address"),

        # ----------------
        # Ordering
        # ----------------
        F.conv(F.col("tx.blockNumber").substr(3, 20), 16, 10)
            .cast("long")
            .alias("block_number"),

        F.conv(F.col("tx.transactionIndex").substr(3, 10), 16, 10)
            .cast("int")
            .alias("tx_index"),

        F.conv(F.col("tx.nonce").substr(3, 10), 16, 10)
            .cast("long")
            .alias("nonce"),

        # ----------------
        # Value & gas
        # ----------------
        F.conv(F.col("tx.value").substr(3, 64), 16, 10)
            .cast("decimal(38,0)")
            .alias("value_wei"),

        F.conv(F.col("tx.gas").substr(3, 20), 16, 10)
            .cast("long")
            .alias("gas_limit"),

        F.conv(F.col("tx.gasPrice").substr(3, 20), 16, 10)
            .cast("long")
            .alias("gas_price"),

        F.conv(F.col("tx.maxFeePerGas").substr(3, 20), 16, 10)
            .cast("long")
            .alias("max_fee_per_gas"),

        F.conv(F.col("tx.maxPriorityFeePerGas").substr(3, 20), 16, 10)
            .cast("long")
            .alias("max_priority_fee_per_gas"),

        # ----------------
        # Call data
        # ----------------
        F.col("tx.input").alias("input_data"),

        # ----------------
        # Type & chain
        # ----------------
        F.col("tx.type").alias("tx_type"),
        F.conv(F.col("tx.chainId").substr(3, 10), 16, 10)
            .cast("int")
            .alias("chain_id"),

        # ----------------
        # Signature
        # ----------------
        F.col("tx.v").alias("sig_v"),
        F.col("tx.r").alias("sig_r"),
        F.col("tx.s").alias("sig_s"),
        F.col("tx.yParity").alias("y_parity")

        # ----------------
        # Raw backup
        # ----------------
        # "raw"
    )
)

In [48]:
silver_tx = (
    silver_tx
    .withColumn(
        "has_input",
        F.when(
            (F.col("input_data").isNull()) | (F.col("input_data") == "0x"),
            F.lit(False)
        ).otherwise(F.lit(True))
    )
)

# method_id
silver_tx = silver_tx.withColumn(
    "method_id",
    F.when(
        F.col("has_input"),
        F.substring(F.col("input_data"), 1, 10)   # 0x + 8 hex
    )
)

# input length
# category: simple transfer, router swap, multicall, weird contract
silver_tx = silver_tx \
    .withColumn("input_length", F.length("input_data")) \
    .withColumn(
        "input_words",
        F.when(
            F.col("has_input"),
            (F.length("input_data") - 10) / 64   # 32 bytes = 64 hex
        )
    )
    
# input hash
# agg, deduplicate
silver_tx = silver_tx.withColumn(
    "input_hash",
    F.when(
        F.col("has_input"),
        F.sha2("input_data", 256)
    )
)

# proxy / router 形态判断
silver_tx = silver_tx.withColumn(
    "is_proxy_like",
    F.when(
        (F.col("input_words") >= 4) &
        (F.col("method_id").isin(
            "0x5ae401dc",  # multicall
            "0x1cff79cd",  # execute
            "0x3593564c"   # universal router
        )),
        F.lit(True)
    ).otherwise(F.lit(False))
)

# add tx_kind - 行为类型
# is_contract_call - 是否调用合约
# fee_model - gas模型
silver_tx = (
    silver_tx
    .withColumn(
        "tx_kind",
        F.when(F.col("to_address").isNull(), "contract_creation")
         .when(F.col("input_data") == "0x", "transfer")
         .otherwise("contract_call")
    )
    .withColumn(
        "is_contract_call",
        F.col("input_data") != "0x"
    )
)

silver_tx = silver_tx.withColumn(
    "fee_model",
    F.when(F.col("tx_type") == "0x2", "eip1559")
     .when(F.col("tx_type") == "0x1", "access_list")
     .otherwise("legacy")
)


In [41]:
spark.sql("""
select is_proxy_like, count(*) 
from silver_transactions
group by is_proxy_like          
""").show()

+-------------+--------+
|is_proxy_like|count(1)|
+-------------+--------+
|         true|   35800|
|        false| 1467299|
+-------------+--------+



In [49]:
silver_tx.printSchema()

root
 |-- block_height: long (nullable = true)
 |-- job_name: string (nullable = true)
 |-- run_id: string (nullable = true)
 |-- kafka_topic: string (nullable = true)
 |-- kafka_partition: integer (nullable = true)
 |-- kafka_offset: long (nullable = true)
 |-- kafka_timestamp: timestamp (nullable = true)
 |-- kafka_date: date (nullable = true)
 |-- tx_hash: string (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- block_number: long (nullable = true)
 |-- tx_index: integer (nullable = true)
 |-- nonce: long (nullable = true)
 |-- value_wei: decimal(38,0) (nullable = true)
 |-- gas_limit: long (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- input_data: string (nullable = true)
 |-- tx_type: string (nullable = true)
 |-- chain_id: integer (nullable = true)
 |-- sig_v: string (

In [50]:
silver_tx.limit(1).toPandas()
# input - 意图
# logs - 结果

,block_height,job_name,run_id,kafka_topic,kafka_partition,kafka_offset,kafka_timestamp,kafka_date,tx_hash,block_hash,from_address,to_address,block_number,tx_index,nonce,value_wei,gas_limit,gas_price,max_fee_per_gas,max_priority_fee_per_gas,input_data,tx_type,chain_id,sig_v,sig_r,sig_s,y_parity,has_input,method_id,input_length,input_words,input_hash,is_proxy_like,tx_kind,is_contract_call,fee_model
0,79064617,realtime_blocks,567750db-f1d8-46d5-96cf-08f5115bc3e3,blockchain.bsc.ingestion.transactions.raw,3,29651,2026-02-03 13:16:54.747,2026-02-03,0x782efac212ce646fdc9e7b7e8a998b782678342115e95beebca9cfb4a30a5058,0x50e462b73140b29efb9ada582e75fbc7ba3f64063b7dff00baafd9b98f515d89,0xee6836e53a7661c7431f6de877cce1faface2ef4,0x278d858f05b94576c1e6f73285886876ff6ef8d2,79064617,4,93237,0,600011,50000001,50000001,50000001,0x42e3dcf30000000000000000000000000000000000000000000000000000004b6b0f1d8b000000000000000000000000ba2ae424d960c26247dd6c32edc70b295c744c43000000000000000000000000000000000000000000000000063d79a508f39c040000000000000000000000000000000000000000000000000000000000000001000000000000000000000000ce6160bb594fc055c943f59de92cee30b8c6b32c000000000000000000000000000000000000000000000000000000006981f5a00d,0x2,56,0x0,0xaeea6fbc469406da3c741fcf45bc67699eccc1791b85522946cd13ada5cb4207,0x5fc30c78d168064c20687ebc0f06a1b2c88a0f7961bf4a45c3f8da05c725a1f5,0x0,True,0x42e3dcf3,396,6.03125,0cfd30d7e57f5a5fb6727ebdbdef15605c15b28068402b2047afdfa53e27a1e0,False,contract_call,True,eip1559


In [56]:
spark.sql("""
select * from silver.bsc_mainnet_transactions where block_height = 79065840 order by tx_index
""").show(truncate=False)

+------------+---------------+------------------------------------+-----------------------------------------+---------------+------------+-----------------------+----------+------------------------------------------------------------------+------------------------------------------------------------------+------------+--------+-------+------------------------------------------+------------------------------------------+----------------+---------+---------+---------------+------------------------+-------+--------+-----+------------------------------------------------------------------+------------------------------------------------------------------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------